In [1]:
import numpy as np
import pandas as pd
import pytse_client as tse
import jdatetime
from datetime import datetime
from scipy.ndimage.interpolation import shift

/tmp/ipykernel_28779/2268364296.py:6: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


# get data from the web

In [3]:
#tickers = tse.download(symbols="all", write_to_csv=True,  include_jdate=True)
Price = pd.read_csv("/home/khaled/Project/AdjustedPrice_P/tickers_data/خودرو.csv")
Price

,date,open,high,low,adjClose,value,volume,count,yesterday,close,jdate
0,2015-02-22,2622.0,2630.0,2580.0,2595.0,26171957431,10084058,1077,2.0,NaN,1393-12-03
1,2015-02-23,2600.0,2669.0,2580.0,2629.0,21321990823,8111076,985,2595.0,2630.0,1393-12-04
2,2015-02-24,2680.0,2681.0,2594.0,2637.0,17145214027,6501084,860,2629.0,2605.0,1393-12-05
3,2015-02-25,2601.0,2641.0,2581.0,2633.0,52358883873,19886628,592,2637.0,2599.0,1393-12-06
4,2015-02-28,2577.0,2630.0,2538.0,2567.0,11375322207,4431642,696,2633.0,2543.0,1393-12-09
...,...,...,...,...,...,...,...,...,...,...,...
1768,2023-03-01,3148.0,3160.0,3033.0,3074.0,1865348981622,606726335,17910,3192.0,3033.0,1401-12-10
1769,2023-03-04,3001.0,3191.0,2985.0,3114.0,2439785513014,783586645,19484,3074.0,3147.0,1401-12-13
1770,2023-03-05,3221.0,3269.0,3170.0,3237.0,2521695929135,779039831,18078,3114.0,3181.0,1401-12-14
1771,2023-03-06,3175.0,3271.0,3115.0,3192.0,1533033944602,480291376,13266,3237.0,3179.0,1401-12-15


In [6]:
#Price = tickers['خودرو']
Price['adjCloseShift'] = shift(Price['adjClose'], 1, cval=np.inf) 
Price

,date,open,high,low,adjClose,value,volume,count,yesterday,close,jdate,adjCloseShift
0,2015-02-22,2622.0,2630.0,2580.0,2595.0,26171957431,10084058,1077,2.0,NaN,1393-12-03,inf
1,2015-02-23,2600.0,2669.0,2580.0,2629.0,21321990823,8111076,985,2595.0,2630.0,1393-12-04,2595.0
2,2015-02-24,2680.0,2681.0,2594.0,2637.0,17145214027,6501084,860,2629.0,2605.0,1393-12-05,2629.0
3,2015-02-25,2601.0,2641.0,2581.0,2633.0,52358883873,19886628,592,2637.0,2599.0,1393-12-06,2637.0
4,2015-02-28,2577.0,2630.0,2538.0,2567.0,11375322207,4431642,696,2633.0,2543.0,1393-12-09,2633.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1768,2023-03-01,3148.0,3160.0,3033.0,3074.0,1865348981622,606726335,17910,3192.0,3033.0,1401-12-10,3192.0
1769,2023-03-04,3001.0,3191.0,2985.0,3114.0,2439785513014,783586645,19484,3074.0,3147.0,1401-12-13,3074.0
1770,2023-03-05,3221.0,3269.0,3170.0,3237.0,2521695929135,779039831,18078,3114.0,3181.0,1401-12-14,3114.0
1771,2023-03-06,3175.0,3271.0,3115.0,3192.0,1533033944602,480291376,13266,3237.0,3179.0,1401-12-15,3237.0


## We want to adjust the price from the 1400-01-01 till the 1401-12-01 with respect to all capital injections that occurred in this interval

In [9]:
FirstDayOfInterval = '1396-01-01'
LastDayOfInterval = '1401-12-01'
def DateT(a):
    return(datetime.fromisoformat((a)))
def DateT1(a):
    return(pd.Timestamp(a))
Price['date'] = Price['date'].apply(lambda x: DateT(x))
gregorian_date = jdatetime.date(int(LastDayOfInterval[0:4]),
                                int(LastDayOfInterval[5:7]),int(LastDayOfInterval[8:10])).togregorian()
Fgregorian_date = jdatetime.date(int(FirstDayOfInterval[0:4]),
                                 int(FirstDayOfInterval[5:7]),int(FirstDayOfInterval[8:10])).togregorian()
PriceFilter = Price[(Price['date'] > pd.Timestamp(pd.to_datetime(Fgregorian_date))) & (Price['date'] < pd.Timestamp(pd.to_datetime(gregorian_date)))]
PriceFilter

,date,open,high,low,adjClose,value,volume,count,yesterday,close,jdate,adjCloseShift
459,2017-03-25,2605.0,2736.0,2605.0,2704.0,18485720279,6836796,679,2606.0,2736.0,1396-01-05,2606.0
460,2017-03-26,2751.0,2798.0,2680.0,2726.0,17212638065,6315207,848,2704.0,2700.0,1396-01-06,2704.0
461,2017-03-27,2720.0,2778.0,2710.0,2754.0,14970985165,5435335,761,2726.0,2739.0,1396-01-07,2726.0
462,2017-03-28,2749.0,2770.0,2710.0,2749.0,14763110525,5369728,771,2754.0,2738.0,1396-01-08,2754.0
463,2017-03-29,2737.0,2780.0,2710.0,2745.0,17582135326,6404212,731,2749.0,2749.0,1396-01-09,2749.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1757,2023-01-28,3080.0,3099.0,2922.0,2990.0,1924390534516,643534786,17583,3031.0,2950.0,1401-11-08,3031.0
1758,2023-01-29,2951.0,2995.0,2841.0,2851.0,2801289007925,982553586,21328,2990.0,2842.0,1401-11-09,2990.0
1759,2023-01-30,2850.0,2925.0,2757.0,2837.0,2018075603422,711228006,17629,2851.0,2913.0,1401-11-10,2851.0
1760,2023-01-31,2898.0,2978.0,2850.0,2931.0,1735417853794,592015718,14282,2837.0,2978.0,1401-11-11,2837.0


In [4]:
Ratio =  PriceFilter['close']/ PriceFilter['yesterday']
CumProd = Ratio.cumprod()
Prod = Ratio.prod()
AdjusetCumProd = CumProd/Prod
PriceFilter['AdjustedPrice'] = AdjusetCumProd*PriceFilter['close'].last()
PriceFilter['AdjustedPrice']

NameError: name 'PriceFilter' is not defined

In [42]:
Price['Ratio'] =  Price['LastPrice']/ Price['LastPriceOfYesterday']
GPrice = Price
aa= Price.groupby('PerasianNameOfCode')
ab = pd.DataFrame(aa['Ratio'].cumprod())
ab = ab.reset_index()
GPrice['Cumprod'] = ab['Ratio']
aaa = Price.groupby('PerasianNameOfCode')
prod = pd.DataFrame(aaa['Ratio'].prod())
prod['LPrice'] = aaa['LastPrice'].last()
prod.rename(columns = {'Ratio':'prod'}, inplace = True)
prod = prod.reset_index()
GPrice = pd.merge(GPrice, prod, on = 'PerasianNameOfCode', how = 'left')
GPrice['Adjust'] = GPrice['LPrice']*(GPrice['Cumprod']/GPrice['prod'])

KeyError: 'LastPrice'

In [ ]:
import jdatetime
from datetime import datetime
FirstDayOfInterval = '1400-08-11'
LastDayOfInterval = '1400-10-20'
def DateT(a):
    return(datetime.fromisoformat((a)))
def DateT1(a):
    return(pd.Timestamp(a))
DataForReqression0['GregorianDate'] = DataForReqression0['GregorianDate'].apply(lambda x: DateT(x))
gregorian_date = jdatetime.date(int(LastDayOfInterval[0:4]),int(LastDayOfInterval[5:7]),int(LastDayOfInterval[8:10])).togregorian()
#dataMerg = dataMerg.fillna(str(pd.Timestamp(gregorian_date)))
DataForReqression1 = DataForReqression0[DataForReqression0['GregorianDate'] > pd.Timestamp(pd.to_datetime(gregorian_date))]
